## IGF Primers - Hands On Session 2
### Quality Control data NGS

Pada tutorial ini kita akan mengevaluasi hasil yang diperoleh dari sequencing menggunakan ONT.

Untuk itu, kita akan mengunduh contoh data yang tersedia dari platform EPI2ME

In [ ]:
%%bash
wget https://ont-exd-int-s3-euwst1-epi2me-labs.s3.amazonaws.com/wf-16s/wf-16s-demo.tar.gz -nc
tar -xzvf wf-16s-demo.tar.gz

Poin diskusi:
- Apa itu proses demultiplexing?
- Ada berapa sampel yang diperoleh dari proses demultiplexing?
- Kenapa ada hasil yang `unclassified`?
- Untuk apa kegunaan md5 sum? 
- Seperti apakah struktur data fastq?

In [ ]:
%%bash
zcat wf-16s-demo/test_data/barcode01/reads1_subset.fastq.gz | head -n 4

Selanjutnya, kita akan melakukan QC dengan menggunakan [NanoPlot](https://github.com/wdecoster/NanoPlot). Tools ini sudah terinstall di resep environment yang disiapkan, namun dapat diinstal juga melalui conda atau pip dengan cara:

```bash
# dengan conda
conda install -c bioconda nanoplot

# atau 

# dengan pip
pip install NanoPlot
```

Kita dapat menggunakan fungsi help untuk melihat semua parameter yang tersedia dari NanoPlot

In [ ]:
%%bash
NanoPlot -h

Untuk nge-running NanoPlot, bisa kita gunakan command berikut:

In [3]:
%%bash
NanoPlot -o barcode01_reads1 --fastq wf-16s-demo/test_data/barcode01/reads1_subset.fastq.gz -t 2

Jika kita perhatikan, ada lebih dari satu file fastq yang ada dari hasil sequencing, Bagaimana cara kita melakukan QC dengan NanoPlot dengan lebih efisien, supaya tidak perlu menulis script berulang-ulang?

Kita bisa menggunakan `for loop` yang tersedia dari UNIX:

In [ ]:
%%bash
#!/bin/bash

# Directory containing the fastq folders
FASTQ_DIR="wf-16s-demo/test_data"
OUTPUT_DIR="output_data"
mkdir -p "$OUTPUT_DIR"

# Iterate over each folder in the fastq directory
for folder in "$FASTQ_DIR"/*; do
  if [ -d "$folder" ]; then
    # Extract the folder name
    folder_name=$(basename "$folder")
    
    # Print message to track progress
    echo "Running NanoPlot for folder: $folder_name"
    
    # Merge FastQ
    temp_fastq="$OUTPUT_DIR/$folder_name.fastq.gz"
    cat "$folder"/*.fastq.gz > "$temp_fastq"

    # Run NanoPlot for the folder
    NanoPlot --fastq "$temp_fastq" -t 2 -o "$OUTPUT_DIR/$folder_name"

    # Print message after completion
    echo "Completed NanoPlot for folder: $folder_name"
  fi
done